(1)

In [10]:
# References
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py
# http://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class
import torch 
import numpy as np 
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader


#데이터 로드와 전처리과정 
#DiabetesDataset 클래스 생성 
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""
    
    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('diabetes.csv.gz',
                        delimiter=',', dtype=np.float32) # 해당 파일로부터 데이터를 불러온다.
        self.len = xy.shape[0] #길이 정보 저장
        self.x_data = torch.from_numpy(xy[:, 0:-1]) #마지막 열 빼고 x_data에 저장
        self.y_data = torch.from_numpy(xy[:, [-1]]) #마지막 열은 class label로 y_data에 저장 

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] # 인덱스에 하나의 아이템 반환 

    def __len__(self):
        return self.len # 데이터 길이 반환 


dataset = DiabetesDataset() #인스탠스 생성 
train_loader = DataLoader(dataset=dataset, #dataset instance
                          batch_size=32, # 배치사이즈 32
                          shuffle=True,  # shuffling 수행
                          num_workers=0) #원래, num_workers = 2  >> subprocess 2개 
                                         # 하지만 2로 하니 runtimeerror가 발생했고 이를 0으로 고치니 작동이 되었음. 


class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6) # 입력층 8개 레이블 , 은닉층 출력 6개 레이블
        self.l2 = torch.nn.Linear(6, 4) #은닉 입력 6, 은닉 입력 4 
        self.l3 = torch.nn.Linear(4, 1) # 은닉 입력 4, 최종 출력층 레이블 1개

        self.sigmoid = torch.nn.Sigmoid() # 활성함수 시그모이드 사용 

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x)) #해당 층에서 전방계산이후 활성함수 적용 
        out2 = self.sigmoid(self.l2(out1)) # 활성함수 적용 
        y_pred = self.sigmoid(self.l3(out2)) # 활성함수 적용 
        return y_pred


# our model
model = Model()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True) #목적함수 바이너리 크로스 엔트로피 사용 
optimizer = torch.optim.SGD(model.parameters(), lr=0.1) # 최적화 스토캐스틱 그래디언드 디센트 사용 

# Training loop
for epoch in range(2): # 총 2 epoch
    for i, data in enumerate(train_loader, 0):
        
        # get the inputs
        inputs, labels = data # 인풋과 레이블 받기 
        
        #wrap them in Variable 
        inputs, labels = Variable(inputs), Variable(labels) # Variable() 감싸기

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs) # 예측된 y 

        # Compute and print loss
        loss = criterion(y_pred, labels) # 목적함수 값 
        print(epoch, i, loss.data)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() #초기화 
        loss.backward() #백프로페게이션 
        optimizer.step() #변수 업데이트 

0 0 tensor(0.9322)
0 1 tensor(0.9841)
0 2 tensor(0.8528)
0 3 tensor(0.8780)
0 4 tensor(0.8929)
0 5 tensor(0.8623)
0 6 tensor(0.7339)
0 7 tensor(0.7703)
0 8 tensor(0.7970)
0 9 tensor(0.8302)
0 10 tensor(0.7377)
0 11 tensor(0.7167)
0 12 tensor(0.7293)
0 13 tensor(0.7030)
0 14 tensor(0.7157)
0 15 tensor(0.7044)
0 16 tensor(0.7013)
0 17 tensor(0.6916)
0 18 tensor(0.6963)
0 19 tensor(0.6901)
0 20 tensor(0.6963)
0 21 tensor(0.6884)
0 22 tensor(0.6807)
0 23 tensor(0.6835)
1 0 tensor(0.6840)
1 1 tensor(0.6672)
1 2 tensor(0.6712)
1 3 tensor(0.6919)
1 4 tensor(0.6689)
1 5 tensor(0.6793)
1 6 tensor(0.6456)
1 7 tensor(0.6463)
1 8 tensor(0.6500)
1 9 tensor(0.6573)
1 10 tensor(0.6560)
1 11 tensor(0.6549)
1 12 tensor(0.6429)
1 13 tensor(0.6526)
1 14 tensor(0.6634)
1 15 tensor(0.6389)
1 16 tensor(0.7005)
1 17 tensor(0.6268)
1 18 tensor(0.6624)
1 19 tensor(0.6361)
1 20 tensor(0.6352)
1 21 tensor(0.6617)
1 22 tensor(0.6477)
1 23 tensor(0.6695)


(2) DataLoader 의 batch_size는 우리가 모델을 학습시킬 때 설정한 배치의 사이즈를 말한다. shuffle은 original data로부터 shuffling을 할 것인지를 묻는 것이고, num_worker 는  데이터를 로드하는데 얼마나 많은 subprocess들을 둘 것인지를 의미한다. 디폴트 값은 0으로 main process에서 데이터를 로드할 것임을 의미한다. 

___________________________
++ 영상내용 

데이터 로드 
현재의 데이터셋은 700개 가량으로 모두 피팅하는데 문제가 없지만 데이터셋이 
많아지면 모두 피팅하기 어려움 

그래서 배치를 씀  

epoch는 모든 트레이닝 데이터가 학습에 한번 forward backword 되는 것을 말하고 
batch size는 트레이닝 데이터를 몇개씩 나눌것인지를 말한다. 
iteration은 할당된 배치 사이즈의 하나의 배치가 학습에 이용되는 것을 말한다. 
예를 들어, 1000개의 트레닝 데이터가 있고 배치사이즈가 500이면 2개의 배치가 만들어지고 에포크 한번을 cover하기 위해 2번의 iteration이 필요하게 된다. 

 